In [ ]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv
from data_cleaning_py import clean_025

load_dotenv()

In [ ]:
headers = {
    "QB-Realm-Hostname": os.getenv("QB_REALMHOSTNAME"),
    "User-Agent": os.getenv("QB_USERAGENT"),
    "Authorization": f"QB-USER-TOKEN {os.getenv('QB_AUTHORIZATION')}",
    "Content-Type": "application/json",
}

Get field ID's

In [ ]:
join_fields = [
    "year_fiscal",
    "agency_id",
    "fund_id",
    "program_id",
    "costcenter_id",
    "account_id",
    "project_id",
    "award_id",
]
query_fields = join_fields + ["qbid"]
insert_fields = join_fields + [
    "account_3_id",
    "account_1_id",
    "budget_adjusted",
    "budget_spent",
    "budget_current",
]
all_fields = insert_fields + ["qbid"]

params = {"tableId": "bu9duyip8"}

r = requests.get(
    url="https://api.quickbase.com/v1/fields", params=params, headers=headers
)

fields = {
    field["label"]: field["id"] for field in r.json() if field["label"] in all_fields
}

field_ids = {v: k for k, v in fields.items()}

field_ids_query = list({v: k for k, v in fields.items() if k in query_fields})

Query for data

In [ ]:
body = {"from": "bu9duyip8", "select": field_ids_query}

r = requests.post(
    url="https://api.quickbase.com/v1/records/query", json=body, headers=headers
)

balances = r.json()["data"]

balances_normalized = {}
for field_id in field_ids_query:
    balances_normalized[field_id] = []
    for balance in balances:
        balances_normalized[field_id].append(balance[str(field_id)]["value"])

balances_normalized = pd.DataFrame(balances_normalized).rename(columns=field_ids)

In [ ]:
len(balances_normalized)

Update records

In [ ]:
r_025 = clean_025.clean_025()

In [ ]:
len(r_025)

In [ ]:
balances = balances_normalized.merge(r_025, how="outer", on=join_fields)

balances

In [ ]:
missing_from_qb = (
    balances.loc[lambda df: df["qbid"].isna()]
    .drop(columns=["qbid"])
    .rename(columns=fields)
    .to_dict(orient="records")
)

len(missing_from_qb)

data = []
for row in missing_from_qb:
    data.append({k: {"value": v} for k, v in row.items()})

In [ ]:
body = {"to": "bu9duyip8", "data": data}

r = requests.post(
    url="https://api.quickbase.com/v1/records", json=body, headers=headers
)

r.json()